In [ ]:
!pip install transformers==4.16.2
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.optim import AdamW
from torch.utils.data import Dataset,DataLoader
from transformers import AutoModel,AutoConfig,AutoTokenizer
from transformers import get_cosine_schedule_with_warmup
from tqdm import tqdm_notebook
import gc
from transformers import DataCollatorWithPadding

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pandas.core.arrays.period import validate_dtype_freq
class Cfg:
    tr_batch_size=1
    val_batch_size=4
    epochs=5
    accumulation_steps=16
    max_len=1024
    
    eps=10**-6
    lr2=5*10**-6
    lr1=10**-3
    betas=(.9,.999)
    model='microsoft/deberta-v3-large'
    batch_schedule=True
    device='cuda' if torch.cuda.is_available() else 'cpu'

In [1]:
!nvidia-smi

Tue Jul 12 01:08:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#train_df=pd.read_csv('/content/drive/MyDrive/saurabh_feed_back_train.csv')
#test_df=pd.read_csv('/content/test.csv')

In [ ]:
import random

In [ ]:
#!pip install torch==1.10
#!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:40tcmalloc: large alloc 1147494400 bytes == 0x66128000 @  0x7f9b56e1a615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████████████████| 881.9 MB 17 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.10.0 which is incom

In [ ]:
import nltk
import re
from nltk.tokenize import sent_tokenize, word_tokenize

In [ ]:
import re
nltk.download('punkt')

def replace_fn(x):
    x=x.replace('\xa0',' ')
    x=x.replace('[','1111111111')
    x=x.replace(']','2222222222')
    x=x.replace('{','3333333333')
    x=x.replace('}','4444444444')
    x=x.replace('(','5555555555')
    x=x.replace(')','6666666666')
    #x=x.replace('.','7777777777')
    x=x.replace('?','8888888888')
    x=x.replace('*','999999999')
    x=x.replace('\\','0000000000')
    x=x.replace('$','xxxxxxxxxx')
    x=x.replace('+','yyyyyyyyyy')
    x=x.replace('^','zzzzzzzzzz')
    x=x.replace('-','aaaaaaaaaa')
    x=x.replace('|','bbbbbbbbbb')
    x=x.replace('!','cccccccccc')
    return x
#train_df['discourse_text']=train_df['discourse_text'].apply(lambda x:replace_fn(x))
#train_df['essay']=train_df['essay'].apply(lambda x:replace_fn(x))

def replace_back(x):
    x=x.replace('1111111111','[')
    x=x.replace('2222222222',']')
    x=x.replace('3333333333','{')
    x=x.replace('4444444444','}')
    x=x.replace('5555555555','(')
    x=x.replace('6666666666',')')
    #x=x.replace('.','7777777777')
    x=x.replace('8888888888','?')
    x=x.replace('999999999','*')
    x=x.replace('0000000000','\\')
    x=x.replace('xxxxxxxxxx','$')
    x=x.replace('yyyyyyyyyy','+')
    x=x.replace('zzzzzzzzzz','^')
    x=x.replace('aaaaaaaaaa','-')
    x=x.replace('bbbbbbbbbb','|')
    x=x.replace('cccccccccc','!')
    return x

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:

max_len=0
l=[]
'''for i,row in enumerate(train_df.to_dict('records')):
    encoding=tokenizer(row['discourse_text'],row['essay'])
    length=len(encoding['input_ids'])
    #print(length)
    l.append(length)
    if length>max_len:
        #print(i)
        
        max_len=length'''
max_len

0

In [ ]:
#train_df['context']=0

In [ ]:
'''
from nltk.tokenize import sent_tokenize
for i,row in enumerate(train_df.to_dict('records')):
    m=re.search(row['discourse_text'].strip(),row['essay'])
    if m!=None:
       
        t1=sent_tokenize(replace_back(row['essay'][0:m.start()]))
        
        t2=sent_tokenize(replace_back(row['essay'][m.end():]))
        text=replace_back(row['essay'][m.start():m.end()])
        x=random.choice([i for i in range(6,26)])
        y=random.choice([i for i in range(6,26)])
        z=random.choice([i for i in range(12,35)])
        if m.start()!=0:
            train_df['context'][i]='.'.join(t1[(len(t1)-x)::])+'. '+text+' '+'.'.join(t2[0:y:])+'.'
        else:
            train_df['context'][i]=text+' '+'.'.join(t2[0:z:])+'.'
            
        train_df['discourse_text'][i]=replace_back(row['discourse_text'])
        train_df['essay'][i]=replace_back(row['essay'])
    else:
        train_df['context'][i]=replace_back(row['essay'])
        train_df['discourse_text'][i]=replace_back(row['discourse_text'])
        train_df['essay'][i]=replace_back(row['essay'])'''

"\nfrom nltk.tokenize import sent_tokenize\nfor i,row in enumerate(train_df.to_dict('records')):\n    m=re.search(row['discourse_text'].strip(),row['essay'])\n    if m!=None:\n       \n        t1=sent_tokenize(replace_back(row['essay'][0:m.start()]))\n        \n        t2=sent_tokenize(replace_back(row['essay'][m.end():]))\n        text=replace_back(row['essay'][m.start():m.end()])\n        x=random.choice([i for i in range(6,26)])\n        y=random.choice([i for i in range(6,26)])\n        z=random.choice([i for i in range(12,35)])\n        if m.start()!=0:\n            train_df['context'][i]='.'.join(t1[(len(t1)-x)::])+'. '+text+' '+'.'.join(t2[0:y:])+'.'\n        else:\n            train_df['context'][i]=text+' '+'.'.join(t2[0:z:])+'.'\n            \n        train_df['discourse_text'][i]=replace_back(row['discourse_text'])\n        train_df['essay'][i]=replace_back(row['essay'])\n    else:\n        train_df['context'][i]=replace_back(row['essay'])\n        train_df['discourse_text']

In [ ]:
tokenizer=AutoTokenizer.from_pretrained(Cfg.model,use_fast=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
class TrainDataset(Dataset):
    def __init__(self,df,tokenizer,max_len):
        self.df=df
        self.max_len=max_len
        self.tokenizer=tokenizer
    def __len__(self):
        return len(self.df)
    def __getitem__(self,item):
        discourse_text=self.df['discourse_text'][item]
        essay=self.df['essay'][item]
        label=self.df['discourse_effectiveness'][item]
        dis_type=self.df['discourse_type'][item]
        
        text=discourse_text+'[SEP]'+essay
        topic=dis_type+'[SEP]'+self.df['topic_name'][item]
        encoding=tokenizer(dis_type,text,truncation=True,max_length=Cfg.max_len)
      
       
        encoding['labels']=label
        return encoding
    
    
class TestDataset(Dataset):
    def __init__(self,df,tokenizer,max_len):
        self.df=df
        self.max_len=max_len
        self.tokenizer=tokenizer
    def __len__(self):
        return len(self.df)
    def __getitem__(self,item):
        discourse_text=self.df['target'][item]
        essay=self.df['essay'][item]
        #label=self.df['discourse_effectiveness'][item]
        encoding=tokenizer(discourse_text,essay,max_length=Cfg.max_len,padding='max_length',
                          truncation=True)
        for k,v in encoding.items():
            encoding[k]=torch.tensor(v,dtype=torch.long)
        #label=torch.tensor(label,dtype=torch.long)
        return encoding

In [ ]:
class Collate:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, batch):
        output = dict()
        output["input_ids"] = [sample["input_ids"] for sample in batch]
        output["attention_mask"] = [sample["attention_mask"] for sample in batch]
        output['labels']=[sample["labels"] for sample in batch]
        output['token_type_ids']=[sample['token_type_ids'] for sample in batch]

        # calculate max token length of this batch
        batch_max = max([len(ids) for ids in output["input_ids"]])
        
        
        
        

        # add padding
        if batch_max<=Cfg.max_len:
            output["input_ids"] = [s[0:-1] + (batch_max - len(s)) * [self.tokenizer.pad_token_id]+[self.tokenizer.sep_token_id] 
                                   for s in output["input_ids"]]
            output["attention_mask"] = [s + (batch_max - len(s)) * [0] for s in 
                                        output["attention_mask"]]
            output['token_type_ids']=[s + (batch_max - len(s)) * [0] for s in 
                                      output["token_type_ids"]]
        else:
            output["input_ids"] = [s[0:Cfg.max_len-1] + [self.tokenizer.sep_token_id] if len(s)>Cfg.max_len else  
                                   s[0:-1] + (Cfg.max_len - len(s)) * [self.tokenizer.pad_token_id]+[self.tokenizer.sep_token_id]
                            
                             for s in output["input_ids"]  ]
            output["attention_mask"] = [ s[0:Cfg.max_len] if len(s)>Cfg.max_len else s + (Cfg.max_len - len(s)) * [0]
                                        for s in output["attention_mask"] ]
            output['token_type_ids']=[s[0:Cfg.max_len]  if len(s)>Cfg.max_len else 
                                  s + (Cfg.max_len - len(s)) * [0]  for s in output["token_type_ids"]  ]



        # convert to tensors
        output["input_ids"] = torch.tensor(output["input_ids"], dtype=torch.long)
        output["attention_mask"] = torch.tensor(output["attention_mask"], dtype=torch.long)
        output['labels']=torch.tensor(output['labels'],dtype=torch.long)
        output['token_type_ids']=torch.tensor(output['token_type_ids'],dtype=torch.long)

        return output

In [ ]:
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings

In [ ]:
from torch.nn.modules.activation import Sigmoid
from torch.nn.modules.activation import Sigmoid
import torch.nn.functional as F
class LongModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.config=AutoConfig.from_pretrained(Cfg.model)
        self.base=AutoModel.from_pretrained(Cfg.model,config=self.config)
        self.lstm=torch.nn.LSTM(1024, 512, batch_first=True,bidirectional=True,num_layers=2)
        self.conv1d_layer1 = torch.nn.Conv1d(1024, 1024, kernel_size=1)
        self.conv1d_layer3 = torch.nn.Conv1d(1024, 1024, kernel_size=3, padding=1)
        self.conv1d_layer5 = torch.nn.Conv1d(1024, 1024, kernel_size=5, padding=2)
        self.class_projector = torch.nn.Sequential(
                torch.nn.LayerNorm(1024*2),
                torch.nn.Linear(1024*2, 3)
            )


        self.fc1=nn.Linear(self.config.hidden_size,3)
        self.fc2=nn.Linear(384,3)
        self.d=nn.Dropout(p=.2)
        self.pooler=MeanPooling()
    
        
        self._init_weights(self.base)
        self._init_weights(self.fc1)
        self._init_weights(self.fc2)
       
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
    def forward(self,inputs):
        transformer_output = self.base(**inputs)[0]
        conv_input = transformer_output.transpose(1, 2) # batch, hidden, seq

        conv_output1 = F.relu(self.conv1d_layer1(conv_input)) 
        conv_output3 = F.relu(self.conv1d_layer3(conv_input)) 
        conv_output5 = F.relu(self.conv1d_layer5(conv_input)) 

        concat_output = torch.cat((conv_output1, conv_output3), dim=1).transpose(1, 2)

        lstm_output, (h,c) = self.lstm(transformer_output)
        hidden = torch.cat((lstm_output[:,-1, :int(self.config.hidden_size/2)],lstm_output[:,0, int(self.config.hidden_size/2):]),dim=-1)
        #out=self.lstm(hidden)
        out = self.pooler(lstm_output, inputs['attention_mask'])
        #out = self.pooler(concat_output, inputs['attention_mask'])
        #out = self.d(out)
        #output = self.class_projector(out)
        
        outputs = self.fc1(out)
        return outputs

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
def train_fn(model,train_loader,optimizer,scheduler,criterion,val_loader,criterion2,epoch):
    model.train()
    device=Cfg.device
    model.to(device)
    losses=AverageMeter()
    preds=[]
    labels=[]
    accu_steps=Cfg.accumulation_steps
    scaler=torch.cuda.amp.GradScaler(enabled=True)
    global best_loss
    global best_score
    global x

    
    

    for batch_idx,(inputs) in tqdm_notebook(enumerate(train_loader),total=len(train_loader)):
        for k,v in inputs.items():
            inputs[k]=v.to(device)
        label=inputs['labels']
        inputs1={}
        for k,v in inputs.items():
          if k!='labels':
            inputs1[k]=v
        with torch.cuda.amp.autocast(enabled=True):
            y_preds=model(inputs1)
        #print(y_preds.shape)
        #print(y_preds)
        loss=criterion(y_preds,label)
        losses.update(loss.item(),Cfg.tr_batch_size)
        scaler.scale(loss).backward()
        
        if (batch_idx+1)%accu_steps==0 or batch_idx+1==len(train_loader):
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            if Cfg.batch_schedule:
                scheduler.step()
        #y_preds=torch.nn.functional.softmax(y_preds,dim=1)
        #y_preds=y_preds.detach().to('cpu').numpy()
        #label=label.detach().to('cpu').numpy()
        #preds.append(y_preds)
        #labels.append(label)


        if ((batch_idx+1)%500==0 and epoch==3) or (((batch_idx+1)%6000==0 or (batch_idx+1)%4000==0) and epoch!=1) or batch_idx+1==len(train_loader):
            val_loss,val_preds,val_labels=val_fn(model,val_loader,criterion2)
            val_preds=[torch.nn.functional.softmax(pred,dim=1).detach().to('cpu').numpy() for pred in val_preds]
            val_labels=[label.detach().to('cpu').numpy() for label in val_labels]
            val_preds=np.concatenate(val_preds)
            val_labels=np.concatenate(val_labels)
            val_score=log_loss(val_labels,val_preds)
            print(f'training and val loss for {fold}th fold, {epoch}th epoch is {losses.avg}, {val_loss}')
            print(f'val score for {fold}th fold, {epoch}th epoch is {val_score}')
            if best_score>val_score :
                print(f'best loss={best_loss},new_loss={val_loss},best_score={best_score},new_score={val_score}')
                torch.save(model,f'/content/drive/MyDrive/feed_back_model_2lstm_{Cfg.max_len}_{fold}_fold_{epoch}_epoch.bin')
                #break

                
                
                best_loss=val_loss
                best_score=val_score
                x=0
            else:
                print('**************************************************')
                x=x+1
                print('x=',x)
            
            torch.cuda.empty_cache()
            
            model.train()
            gc.collect()
        if x==14:
                break
            

        


    #predictions=np.concatenate(preds)
    #labels=np.concatenate(labels)
    return losses.avg


def val_fn(model,val_loader,criterion):
    model.eval()
    device=Cfg.device
    losses=AverageMeter()
    preds=[]
    labels=[]
    model.to(device)

    for batch_idx,(inputs) in tqdm_notebook(enumerate(val_loader),total=len(val_loader)):
        for k,v in inputs.items():
            inputs[k]=v.to(device)
        label=inputs['labels']
        inputs1={}
        for k,v in inputs.items():
          if k!='labels':
            inputs1[k]=v
        with torch.no_grad():
            y_preds=model(inputs1)
        loss=criterion(y_preds,label)
        
        losses.update(loss.item(),Cfg.val_batch_size)

        #y_preds=torch.nn.functional.softmax(y_preds,dim=1)
        #y_preds=y_preds.detach().to('cpu').numpy()
        #label=label.detach().to('cpu').numpy()
        preds.append(y_preds)
        labels.append(label)
    #predictions=np.concatenate(preds)
    #labels=np.concatenate(labels)
    return losses.avg,preds,labels
    

In [ ]:
def prepare_input(df,fold):


  train_data=df[df['fold']!=fold].reset_index(drop=True)
  val_data=df[df['fold']==fold].reset_index(drop=True)
  def tokenize_text(type,discourse_text,essay):
    
    t=tokenizer(type,discourse_text+'[SEP]'+essay)
    return len(t['input_ids'])

  val_data['length']=0
  for i,row in enumerate(val_data.to_dict('records')):
    val_data['length'][i]=tokenize_text(row['discourse_type'],row['discourse_text'],row['essay'])

  val_data1=val_data.sort_values('length').reset_index(drop=True)
  val_data=val_data1.copy()
  train_dataset=TrainDataset(train_data,tokenizer,Cfg.max_len)
  val_dataset=TrainDataset(val_data,tokenizer,Cfg.max_len)
  train_dataset=TrainDataset(train_data,tokenizer,Cfg.max_len)
  val_dataset=TrainDataset(val_data,tokenizer,Cfg.max_len)
  
  
  x=len(train_data)
  w0=1-(len(train_data[train_data['discourse_effectiveness']==0])/x)
  w1=1-(len(train_data[train_data['discourse_effectiveness']==1])/x)
  w2=1-(len(train_data[train_data['discourse_effectiveness']==2])/x)
  tr_weights=torch.tensor([w0,w1,w2],dtype=torch.half)
  tr_weights=tr_weights.to('cuda')
  val_weights=torch.tensor([w0,w1,w2],dtype=torch.float)
  val_weights=val_weights.to('cuda')
  #collate_fn = DataCollatorWithPadding(tokenizer)
  collate_fn=Collate(tokenizer)
  #train_loader = DataLoader(train_ds, collate_fn=collate_fn)
  train_loader=DataLoader(train_dataset,collate_fn=collate_fn,shuffle=True,batch_size=Cfg.tr_batch_size,pin_memory=True,num_workers=16)
  val_loader=DataLoader(val_dataset,collate_fn=collate_fn,batch_size=Cfg.val_batch_size,pin_memory=True,num_workers=16)
  
  num_training_steps=int(len(train_loader)*Cfg.epochs/Cfg.accumulation_steps)
  return train_loader,val_loader,num_training_steps,tr_weights,val_weights

In [ ]:
from sklearn.metrics import log_loss

In [ ]:
def train_loop(df,fold):
    model=LongModel()
    #model=torch.load('/content/drive/MyDrive/feed_back_model_1024_2_fold_1_epoch.bin')

   
    train_loader,val_loader,steps,tr_w,val_w=prepare_input(df,fold)
    optimizer=AdamW(model.parameters(),eps=Cfg.eps,betas=Cfg.betas,lr=Cfg.lr2)
    train_loader,val_loader,steps,tr_w,val_w=prepare_input(df,fold)
    #optimizer=AdamW(model.parameters(),eps=Cfg.eps,betas=Cfg.betas)
    criterion1=torch.nn.CrossEntropyLoss(reduction='mean',weight=tr_w)
    criterion2=torch.nn.CrossEntropyLoss(reduction='mean',weight=val_w)
    #criterion=torch.nn.NLLLoss(reduction='mean')
    scheduler=get_cosine_schedule_with_warmup(optimizer,num_warmup_steps=1000,num_training_steps=steps)
    global best_loss
    global best_score
    global x
    global l
    best_loss=1000
    best_score=1000
    x=0
    l=[]
    
    for epoch in range(1,Cfg.epochs+1):
        tr_loss=train_fn(model,train_loader,optimizer,scheduler,
                                            criterion1,val_loader,criterion2,epoch)
        
        #tr_score=log_loss(tr_labels,tr_preds)
        
        

In [ ]:
train_df=pd.read_csv('/content/drive/MyDrive/saurabh_feedback_prize.csv')


In [ ]:
topics=pd.read_csv('/content/topics.csv')
folds=pd.read_csv('/content/groupkfold.csv')
train_df=pd.merge(train_df,folds,on='discourse_id')
train_df=pd.merge(train_df,topics,on='essay_id')


from text_unidecode import unidecode
from typing import Dict, List, Tuple
import codecs

def replace_encoding_with_utf8(error: UnicodeError) -> Tuple[bytes, int]:
    return error.object[error.start : error.end].encode("utf-8"), error.end


def replace_decoding_with_cp1252(error: UnicodeError) -> Tuple[str, int]:
    return error.object[error.start : error.end].decode("cp1252"), error.end

# Register the encoding and decoding error handlers for `utf-8` and `cp1252`.
codecs.register_error("replace_encoding_with_utf8", replace_encoding_with_utf8)
codecs.register_error("replace_decoding_with_cp1252", replace_decoding_with_cp1252)

def resolve_encodings_and_normalize(text: str) -> str:
    """Resolve the encoding problems and normalize the abnormal characters."""
    text = (
        text.encode("raw_unicode_escape")
        .decode("utf-8", errors="replace_decoding_with_cp1252")
        .encode("cp1252", errors="replace_encoding_with_utf8")
        .decode("utf-8", errors="replace_decoding_with_cp1252")
    )
    text = unidecode(text)
    return text

train_df['discourse_text'] = train_df['discourse_text'].apply(lambda x : resolve_encodings_and_normalize(x))
train_df['essay'] = train_df['essay'].apply(lambda x : resolve_encodings_and_normalize(x))
train_df['discourse_effectiveness']=train_df['discourse_effectiveness'].map({'Ineffective':0,'Adequate':1,'Effective':2})

#train_df['discourse_text']=train_df['discourse_text'].apply(lambda x: x.lower())
#train_df['discourse_type']=train_df['discourse_type'].apply(lambda x: x.lower())
#train_df['essay']=train_df['essay'].apply(lambda x: x.lower())
#train_df['topic_name']=train_df['topic_name'].apply(lambda x: x.lower())

In [ ]:
#train_df=train_df.sample(n=len(train_df)).reset_index(drop=True)

In [ ]:
import transformers
transformers.logging.set_verbosity_error()

for fold in range(3,4):
    train_loop(train_df,fold)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/29412 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:94: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1839 [00:00<?, ?it/s]

training and val loss for 3th fold, 1th epoch is 0.7453500246265607, 0.7140047187598391
val score for 3th fold, 1th epoch is 0.6692579390361295
best loss=1000,new_loss=0.7140047187598391,best_score=1000,new_score=0.6692579390361295


...

  0%|          | 0/1839 [00:00<?, ?it/s]

training and val loss for 3th fold, 3th epoch is 0.49582391357421873, 0.7482906118553899
val score for 3th fold, 3th epoch is 0.6863855584537871
**************************************************
x= 2


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:94: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1839 [00:00<?, ?it/s]

training and val loss for 3th fold, 3th epoch is 0.48539786783854166, 0.7408422317078348
val score for 3th fold, 3th epoch is 0.6796902668318238
**************************************************
x= 3
